<a href="https://colab.research.google.com/github/arumm25/Repo1-PenambanganData/blob/main/Repository3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd

path = '/content/drive/MyDrive/Penambangan Data/dataKasus-1.xlsx'
df = pd.read_excel(path)

df.head()


,NO,NAMA,USIA,PARITAS,JARAK KELAHIRAN,RIW HIPERTENSI,RIW PE,OBESITAS,RIW DM,RIW HIPERTENSI/PE DALAM KELUARGA,SOSEK RENDAH,PE/Non PE,Unnamed: 12
0,1,NAMA 1,23 TH,3,< 2 tahun,Tidak,Tidak,Tidak,Ya,Tidak,>UMR,Non PE,NaN
1,2,NAMA 2,29 TH,2,< 2 tahun,Tidak,PEB,Tidak,Tidak,Ada,>UMR,PE,NaN
2,3,NAMA 3,20 TH,1,anak pertama,Tidak,Tidak,Tidak,Tidak,Tidak,>UMR,Non PE,NaN
3,4,NAMA 4,18 TH,1,anak pertama,Tidak,Tidak,Tidak,Tidak,Tidak,>UMR,Non PE,NaN
4,5,NAMA 5,34 TH,3,> 2 tahun,Tidak,Tidak,Tidak,Tidak,Tidak,>UMR,Non PE,NaN


In [3]:
df.isnull().sum()


,0
NO,0
NAMA,0
USIA,3
PARITAS,0
JARAK KELAHIRAN,7
RIW HIPERTENSI,0
RIW PE,0
OBESITAS,0
RIW DM,0
RIW HIPERTENSI/PE DALAM KELUARGA,0


In [4]:
df = df.dropna()   # bisa juga fillna(mean/median)


In [9]:
from sklearn.preprocessing import LabelEncoder

# Preprocess 'USIA' column to extract numeric values and convert to int
# This ensures 'USIA' is treated as a numerical column, not an object for LabelEncoder
df['USIA'] = df['USIA'].astype(str).str.extract(r'(\d+)').astype(int)

le = LabelEncoder()
for col in df.select_dtypes(include='object'):
    df[col] = le.fit_transform(df[col])

In [37]:
X = df.drop('PE/Non PE', axis=1)   # ganti sesuai kolom targetmu
y = df['PE/Non PE']

In [44]:
from sklearn.model_selection import train_test_split

X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(
    X_res, y_res,
    test_size=0.2,
    random_state=42,
    stratify=y_res
)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)


In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [14]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(
    criterion='gini',
    max_depth=5,
    random_state=42
)
model.fit(X_train, y_train)


DecisionTreeClassifier(max_depth=5, random_state=42)

In [15]:
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:,1]


In [16]:
from sklearn.tree import export_text

rules = export_text(model, feature_names=list(X.columns))
print(rules)


|--- NAMA <= 0.67
|   |--- class: 0
|--- NAMA >  0.67
|   |--- NO <= 1.01
|   |   |--- class: 1
|   |--- NO >  1.01
|   |   |--- class: 2



In [17]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.5


In [19]:
from sklearn.metrics import accuracy_score

y_pred_dt = model.predict(X_test)
# y_pred_rf = rf.predict(X_test_scaled) # rf and X_test_scaled are not defined

print("Accuracy DT :", accuracy_score(y_test, y_pred_dt))
# print("Accuracy RF :", accuracy_score(y_test, y_pred_rf))

Accuracy DT : 0.5


In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.00      0.00      0.00         0

    accuracy                           0.50         2
   macro avg       0.50      0.25      0.33         2
weighted avg       1.00      0.50      0.67         2



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [31]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
import numpy as np

skf = StratifiedKFold(n_splits=5)

for i, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print(f"Fold {i+1}")
    print(classification_report(y_test, y_pred))

Fold 1
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

Fold 2
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.00      0.00      0.00         0

    accuracy                           0.50         2
   macro avg       0.50      0.25      0.33         2
weighted avg       1.00      0.50      0.67         2

Fold 3
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2

Fold 4
              precision    recall  f1-score  

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_d

In [40]:
df['PE/Non PE'] = df['PE/Non PE'].replace({2:1})

In [39]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42, k_neighbors=1) # Set k_neighbors to 1 to accommodate minority class with 2 samples
X_res, y_res = sm.fit_resample(X, y)

In [45]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Retrain model on resampled data
model.fit(X_train_resampled, y_train_resampled)

# Make predictions on resampled test data
y_pred_dt_resampled = model.predict(X_test_resampled)

rmse_dt = np.sqrt(mean_squared_error(y_test_resampled, y_pred_dt_resampled))
# rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf)) # rf and y_pred_rf are not defined

print("RMSE DT (Resampled Data):", rmse_dt)
# print("RMSE RF :", rmse_rf)


RMSE DT (Resampled Data): 0.0


In [47]:
import joblib

joblib.dump(model, '/content/drive/MyDrive/model/dt_preprocessing.pkl')
# joblib.dump(rf, '/content/drive/MyDrive/model/rf_preprocessing.pkl') # rf is not defined
joblib.dump(scaler, '/content/drive/MyDrive/model/scaler.pkl')

['/content/drive/MyDrive/model/scaler.pkl']

In [48]:
dt_model = joblib.load('/content/drive/MyDrive/model/dt_preprocessing.pkl')
